In [26]:
from time import sleep
from UN import *

### UN library default
함수
1. UN_availability   
*변수: data_type, frequency, reporting_area, time_period, classification
  
 
2. UN_comtrade  
*human_readable=False, verbose=True,
    period='recent', frequency='A', reporter=842, partner='all', product='total', tradeflow=2

In [27]:
##country_code_setting
country_code_byCode = country_code_setting(True)
country_code_byLetter = country_code_setting(False)

In [5]:
#HS classification call
HS = get_class_info()

KeyboardInterrupt: 

## Extracting coffee trade data
#### HS code  
1. 생두( not roasted or decaffeinated ): 090111  
2. roasted( roasted, not decaffeinated ): 090121

In [6]:
def UN_comtrade(human_readable=False, verbose=True,
    period='recent', frequency='A', reporter=842, partner='all', product='total', tradeflow=2):
    
    global country_code_byLetter
    base_url = 'https://comtrade.un.org/api/get?'
    
    ##조건
    fmt = 'csv' if human_readable else 'json'
    head = 'H' if human_readable else 'M'
    
    if type(reporter) == str:
        reporter = country_code_byLetter[reporter]
    
    parameters = {
            'ps': period,
            'freq': frequency,
            'r': reporter,
            'p': partner,
            'cc': product,
            'rg': tradeflow,
            'px': 'HS',      # Harmonized System (as reported) as classification scheme
            'type': 'C',     # Commodities ('S' for Services)
            'fmt': fmt,      # format of the output
            'max': 50000,    # maximum number of rows -> what happens if number of rows is bigger?
                             # https://comtrade.un.org/data/dev/portal#subscription says it is 100 000
            'head': head     # human readable headings ('H') or machine readable headings ('M')
        }
    
    url = base_url + dict_to_string(parameters)
    print('url :', url)
    
    r = requests.get(url)
    html = r.text
    js = json.loads(html)
    
    if js['validation']['message']:
        print(js['validation']['message'])
    
    print('\n# of data is', js['validation']['count']['value'])
    
    return js 

In [7]:
##data: 1991 ~ 2017
annual_list = list(range(1991, 2018))

In [8]:
countries = list(country_code_byLetter.keys())[1:]

### 수출 모으기
#### Firstly, 생두

In [9]:
##df 기본틀 세우기
bean_export = pd.DataFrame()

In [14]:
###수출국 모으기
exporters = {}

for time in annual_list:
    exporters[time] = set()


In [10]:
errors = []

In [15]:
countries[29]

'Brazil'

In [22]:
country_code_byLetter['Brazil']

'76'

In [12]:
countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belgium-Luxembourg',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bonaire',
 'Bosnia Herzegovina',
 'Botswana',
 'Br. Virgin Isds',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Isds',
 'Central African Rep.',
 'Chad',
 'Chile',
 'China',
 'China, Hong Kong SAR',
 'China, Macao SAR',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Isds',
 'Costa Rica',
 "Côte d'Ivoire",
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czechia',
 'Czechoslovakia',
 "Dem. People's Rep. of Korea",
 'Dem. Rep. of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Rep.',
 'East and West Pakistan',
 'Ecuador',
 'Egypt',
 'El Salva

In [19]:
temp = UN_comtrade(reporter = countries[29], partner = 76,product = '090111', period = 'ALL', tradeflow = 1)


url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=76&p=76&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 6


In [20]:
bean_export = pd.concat([bean_export, pd.DataFrame(temp['dataset'])])

In [21]:
bean_export

,pfCode,yr,period,periodDesc,aggrLevel,IsLeaf,rgCode,rgDesc,rtCode,rtTitle,...,qtAltCode,qtAltDesc,TradeQuantity,AltQuantity,NetWeight,GrossWeight,TradeValue,CIFValue,FOBValue,estCode
0,H5,2017,2017,2017,6,1,1,Import,76,Brazil,...,None,,4018,None,4018,None,3185,None,None,0
1,H5,2018,2018,2018,6,1,1,Import,76,Brazil,...,None,,211,None,211,None,7604,None,None,0
2,H4,2015,2015,2015,6,1,1,Import,76,Brazil,...,None,,56219,None,56219,None,24258,None,None,0
3,H2,2002,2002,2002,6,1,1,Import,76,Brazil,...,None,,266,None,266,None,1161,None,None,0
4,H2,2003,2003,2003,6,1,1,Import,76,Brazil,...,None,,1010,None,1010,None,3989,None,None,0
5,H2,2005,2005,2005,6,1,1,Import,76,Brazil,...,None,,21600,None,21600,None,49076,None,None,0


In [30]:
for ctr in range(len(countries)):
    print('\n********country: ',countries[ctr])

#     try:
    temp = UN_comtrade(reporter = countries[ctr], partner = int(country_code_byLetter[countries[ctr]]),product = '090111', period = 'ALL', tradeflow = 1)
    bean_export = pd.concat([bean_export, pd.DataFrame(temp['dataset'])])

    sleep(40)
#     except:
#         print('#################ERROR###################### ======>  country: {}'.format(countries[ctr]))
#         errors.append([countries[ctr]])
#         sleep(40)
        
    if ctr % 10 == 0:
        bean_export.to_csv('afterStop_bean_export{}.csv'.format(ctr))
        print('######SAVED#####       ctr: '.format(ctr))


********country:  Afghanistan
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=4&p=4&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0
######SAVED#####       ctr: 

********country:  Albania
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=8&p=8&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Algeria
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=12&p=12&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Andorra
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=20&p=20&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Angola
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=24&p=24&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Anguilla
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=660&p=660&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********countr


# of data is 0

********country:  Côte d'Ivoire
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=384&p=384&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0
######SAVED#####       ctr: 

********country:  Croatia
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=191&p=191&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Cuba
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=192&p=192&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Curaçao
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=531&p=531&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Cyprus
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=196&p=196&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Czechia
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=203&p=203&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# 


# of data is 0

********country:  Germany
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=276&p=276&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Ghana
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=288&p=288&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0
######SAVED#####       ctr: 

********country:  Gibraltar
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=292&p=292&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Greece
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=300&p=300&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Greenland
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=304&p=304&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Grenada
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=308&p=308&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# o


# of data is 0

********country:  Mayotte
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=175&p=175&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Mexico
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=484&p=484&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0
######SAVED#####       ctr: 

********country:  Mongolia
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=496&p=496&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Montenegro
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=499&p=499&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Montserrat
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=500&p=500&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Morocco
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=504&p=504&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M


# of data is 0

********country:  San Marino
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=674&p=674&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0

********country:  Sao Tome and Principe
url : https://comtrade.un.org/api/get?ps=ALL&freq=A&r=678&p=678&cc=090111&rg=1&px=HS&type=C&fmt=json&max=50000&head=M

# of data is 0


KeyboardInterrupt: 

In [9]:
countries[230:]

['Tonga',
 'Trinidad and Tobago',
 'Tunisia',
 'Turkey',
 'Turkmenistan',
 'Turks and Caicos Isds',
 'Tuvalu',
 'Uganda',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom',
 'United Rep. of Tanzania',
 'Uruguay',
 'US Virgin Isds',
 'USA',
 'USA (before 1981)',
 'Uzbekistan',
 'Vanuatu',
 'Venezuela',
 'Viet Nam',
 'Wallis and Futuna Isds',
 'Yemen',
 'Zambia',
 'Zimbabwe']

In [31]:
bean_export.to_csv('pc_final_me.csv'.format(ctr))

### 2012년도만 해보기

In [ ]:
for ctr in range(len(countries)):
    print('\n********country: ',countries[ctr])

#     try:
    temp = UN_comtrade(reporter = countries[ctr],product = '090111', period = 2012, tradeflow = 2)
    bean_export = pd.concat([bean_export, pd.DataFrame(temp['dataset'])])
    print('{}/{} clear'.format(ctr, len(countries)))
    
    sleep(37)
#     except:
#         print('#################ERROR###################### ======>  country: {}'.format(countries[ctr]))
#         errors.append([countries[ctr]])
#         sleep(40)
        
    if ctr % 10 == 0:
        bean_export.to_csv('afterStop_bean_export{}.csv'.format(ctr))
        print('######SAVED#####       ctr: '.format(ctr))